## 词向量

In [20]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import codecs
import re
import jieba
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence, Text8Corpus
import multiprocessing

In [21]:
rootPath = '/home/aid/Github/synonym_detection'#os.path.dirname(os.getcwd())
dataPath = rootPath + '/data'
print(rootPath)

/home/aid/Github/synonym_detection


### 数据获取

In [22]:
import pymysql
import re
import json

def get_mysql_data():
    '''description: 获取mysql数据
    '''
    connection = pymysql.connect(host='192.168.100.50', port=3306, user='root', password='Aid@pro888888', db='nlp_tagging', charset='utf8mb4')
    cursor = connection.cursor()
    cursor.execute("SELECT doc_content FROM `aid-tagging`.document")
    data = cursor.fetchall()
    cursor.close()
    connection.close()

    return data


doc_data = get_mysql_data()
with open(dataPath+'/dk_mainsuit_800w.txt', 'w') as f:
    for doc in doc_data:
        try:
            doc_v = doc[0].replace('\n', '').replace('\r', '').replace(' ','').strip()
            f.write(doc_v + '\n')
        except:
            pass
f.close()
print(len(doc_data))

7164844


### 数据清洗

In [23]:
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"""[^a-zA-Z0-9\u4E00-\u9FA5]|[a-zA-Z0-9'!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘'！[\\]^_`{|}~\s]+|[\001\002\003\004\005\006\007\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a]+""")
    line = rule.sub('',line)
    return line

def split_sentence(sentence):
    '''description: 分句
    '''    
    re_sentencesp = re.compile('([﹒﹔﹖﹗．；。！？]["’”」』]{0,2}|：(?=["‘“「『]{1,2}|$))')
    s = sentence
    slist = []
    for i in re_sentencesp.split(s):
        if re_sentencesp.match(i) and slist:
            slist[-1] += i
        elif i:
            slist.append(i)
    return slist

In [24]:
file_reader = codecs.open(dataPath+'/dk_mainsuit_800w.txt', 'r', 'utf-8',errors="ignore")
source_lines = file_reader.readlines()
file_reader.close()

stop_keyword = [line.strip() for line in codecs.open(dataPath+'/stopword_zh.txt','r','utf-8').readlines()]

source_lines = [remove_punctuation(w) for w in source_lines]
source_cut = [" ".join(jieba.cut(w)) for w in source_lines if w not in stop_keyword]

output = codecs.open(rootPath+'/word2vec/separated_file.txt', 'w', 'utf-8')
for line in source_cut:
    output.write(line)
output.close()
print(len(source_cut), source_cut[0])

7164844 患者 主诉 发热 天 咳嗽 天现 病史 天前 患儿 无 明显 诱因 出现 发热 热峰 为 伴有 寒战 无 抽搐 予 退热药 后 体温 可降 至 正常 偶有 咳嗽 为 刺激性 干咳 无 喘息 无 气促 及 呼吸困难 无 烦躁不安 无 皮疹 门诊 查 血常规 门诊 考虑 为 脓 毒血症 门诊 予美洛 西林 舒巴坦 静滴 天 抗感染 治疗 后 病情 未见 明显好转 今为 进一步 治疗 来 我院 就诊 门诊 以脓 毒血症 收入 院自 发病 以来 病人 精神状态 一般 体力 情况 一般 食欲 食量 一般 睡眠 情况 一般 既往 史 否认 肝炎 结核 传染病 史 否认 手术 外伤 史 否认 输血 史 按 当地 防疫 部门 要求 预防接种 传染病 史 否认 肝炎 结核 等 传染病 史 预防接种 史 乙肝 卡介苗 白百破 脊髓灰质炎 麻疹 乙脑 其他 部分 接种 手术 史因 左侧 腹股沟 斜 疝 右侧 鞘 状 突未闭 在 我院 普外科 住院治疗 予行 腹腔镜 左侧 腹股沟 斜 疝 修补术 右侧 鞘 状 突未闭 修补术 输血 史 既往 因 贫血 输血 治疗 次 过敏史 对 牛奶 牛肉 海鲜 及 西红柿 过敏 个人 史 出生 史 第胎 第产 孕期 周 分娩 因 羊水 过少行 剖宫产 出生 体重 克 出生 时 情况 无 窒息 产伤 史母 孕期 健康 体健 家族史 父亲 年龄 岁 健康状况 健康 母亲 年龄 岁 健康状况 健康 母曾 孕胎 流产 胎 人流 胎 死胎 其他 婚育 史 月经 史 婚姻 史 生育 史 月经 史 初潮 岁 天天 末次 月经


### 模型训练

In [25]:
#source_cut:分好词的文本
#outp1:训练好的模型
#outp2:得到的词向量
model = Word2Vec(LineSentence(rootPath+'/word2vec/separated_file.txt'), vector_size=200, window=5, min_count=5, workers=multiprocessing.cpu_count())
model.save(rootPath+'/word2vec/separated.model')
#不以C语言可以解析的形式存储词向量
model.wv.save_word2vec_format(rootPath+'/word2vec/separated.vector', binary=False)

### 可视化图

In [ ]:
def tsne_plot(model):
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(18, 18)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
    
tsne_plot(w2v_model)

In [ ]:
model_wv_df = pd.DataFrame(model[model.wv.vocab], list(model.wv.vocab))
keywords = ["行情", "旅游", "手机", "股权","基金", "招聘", "高考", "分析","录取", \
            "学生", "员工", '军事', '老板', '战争','国际', '服务',"健康","地区",\
           "俄罗斯","台湾","中国","董事会","风险","部队"]
words = [word for word in keywords if word in list(model.wv.vocab)]

X = model_wv_df.T[words].T
pca = PCA(n_components=2)
result = pca.fit_transform(X)

df = pd.DataFrame(result, columns=["Component 1", "Component 2"])
df["Word"] = keywords
df["Distance"] = np.sqrt(df["Component 1"]**2 + df["Component 2"]**2)
fig = px.scatter(df, x="Component 1", y="Component 2", text="Word", color="Distance", color_continuous_scale="agsunset",size="Distance")
fig.update_traces(textposition='top center')
fig.layout.xaxis.autorange = True
fig.data[0].marker.line.width = 1
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.update_layout(height=800,  template="plotly_white", paper_bgcolor="#f0f0f0")
fig.show()

## 预测

In [2]:
import gensim
rootPath = '/home/aid/Github/synonym_detection'

model = gensim.models.Word2Vec.load(rootPath+'/word2vec/separated.model')
model.wv.most_similar(positive=['未见'], topn=100)

[('内未见', 0.6911172866821289),
 ('余未见', 0.566469669342041),
 ('区未见', 0.563393235206604),
 ('影未见', 0.5555785894393921),
 ('示未见', 0.5357312560081482),
 ('暂未见', 0.5343140363693237),
 ('无', 0.47653254866600037),
 ('腺未见', 0.43761494755744934),
 ('叶未见', 0.4373205006122589),
 ('下未见', 0.433382511138916),
 ('超未见', 0.431390643119812),
 ('处未见', 0.43025442957878113),
 ('胰未见', 0.41962721943855286),
 ('腹未见', 0.41961655020713806),
 ('骨未见', 0.4077603220939636),
 ('整未见', 0.40723374485969543),
 ('灶未见', 0.39857786893844604),
 ('扫未见', 0.3926079571247101),
 ('未见异常', 0.38501209020614624),
 ('现未见', 0.38494664430618286),
 ('率未见', 0.38264599442481995),
 ('项未见', 0.3762330710887909),
 ('可未见', 0.3759281039237976),
 ('见', 0.3735075891017914),
 ('肾未见', 0.3721506595611572),
 ('支未见', 0.37003573775291443),
 ('未示', 0.36812785267829895),
 ('片未见', 0.36710235476493835),
 ('余全', 0.36054834723472595),
 ('值未见', 0.3577535152435303),
 ('征象', 0.3547970950603485),
 ('影无', 0.3529451787471771),
 ('团未见', 0.351279079914093),
 ('端未见',

In [ ]:
model.wv.similarity('发热', '高热')

## 文本相似度

### 字面相似度计算

In [ ]:
def get_chars_sim_score(str1, str2):
    """字面相似度计算"""
    def same_chars(str1, str2):
        return set(str1) & set(str2)

    dp = min(len(str1) / len(str2), len(str2) / len(str1))
    alpha = 0.6
    beta = 0.4
    sames = same_chars(str1, str2)
    def get_weighted(word, sames):
        top = 0.0
        bottom = 0.0
        for idx, i in enumerate(word):
            if i in sames:
                top += (idx + 1)
            bottom += (idx + 1)
        return top / bottom

    p1 = alpha * (len(sames) / len(str1) + len(sames) / len(str2)) / 2
    p2 = beta * dp * (get_weighted(str1, sames) + get_weighted(str2, sames)) / 2
    score = p1 + p2
    return score, sames

In [ ]:
get_chars_sim_score('想你', '不想你')

### 句子计算

In [3]:
import jieba
import numpy as np

def encode(model, sentences):
    all_embeddings = []
    for sentence in sentences:
        emb = []
        count = 0
        for word in jieba.cut(sentence):
            # 调用词向量
            if word in model.wv.key_to_index:
                emb.append(model.wv.get_vector(word, norm=True))
                count += 1
            else:
                if len(word) == 1:
                    continue
                
        tensor_x = np.array(emb).sum(axis=0)  # 纵轴相加
        if count > 0:
            avg_tensor_x = np.divide(tensor_x, count)
        else:
            avg_tensor_x = 0.0
        all_embeddings.append(avg_tensor_x)
    all_embeddings = np.array(all_embeddings)
    return all_embeddings


def try_divide(x, y, val=0.0):
    """
    try to divide two numbers
    """
    if y != 0.0:
        val = float(x) / y
    return val


def cosine_distance(v1, v2):
    """
    余弦距离
    return cos score
    """
    up = float(np.sum(v1 * v2))
    down = np.linalg.norm(v1) * np.linalg.norm(v2)
    return try_divide(up, down)

In [13]:
e1 = encode(model, ['双侧大小脑半球灰白质界限清楚'])
e2 = encode(model, ['两侧脑实质未见明显异常信号影'])
cos_score = cosine_distance(e1, e2)
cos_score

0.6186868791128292

## 其它

In [29]:
import jieba
import jieba.analyse

text = "无明显紫绀，非癫痫事件，癫痫较前无明显降低, 不是，非，不可，未见"
text_cut = jieba.posseg.cut(text)
text_wf = []
for w in text_cut:
    text_wf.append(w.word +'/'+ w.flag + ' ')
print("".join(text_wf))

无/v 明显/a 紫绀/n ，/x 非/h 癫痫/n 事件/n ，/x 癫痫/n 较/d 前无/i 明显降低/n ,/x  /x 不是/c ，/x 非/h ，/x 不可/v ，/x 未/d 见/v 


In [16]:
text = """全身皮肤粘膜无黄染、紫绀，无皮疹、皮下出血。毛发分布正常，皮下无水肿，无肝掌、蜘蛛痣。  全身浅表淋巴结包括耳前、耳后、枕后、颈后、颈前、颌下、颏下、锁骨上、腘窝、滑车、腹股沟等淋巴结未触及肿大。  头颅大小正常，无畸形、压痛。双侧眼睑无水肿及下垂，无倒睫，结膜无充血、水肿，眼球运动自如，双侧巩膜无黄染，双侧瞳孔等大等圆，对光反射灵敏。双侧耳廓无畸形，无耳瘘，外耳道无异常分泌物，双侧乳突无压痛，双侧听力粗试无障碍。鼻外观无畸形，无鼻翼扇动、鼻塞及分泌物，鼻窦无压痛，双侧鼻唇沟对称。口唇无紫绀、苍白，口腔粘膜无溃疡、出血点，伸舌无偏斜、震颤，齿龈无肿胀、溢脓、出血，咽部粘膜无充血，双侧扁桃体无肿大。  颈部对称，无抵抗，颈动脉搏动正常，颈静脉无怒张，气管居中，肝颈静脉回流征阴性，甲状腺无肿大，无压痛、震颤、血管杂音。  胸廓无畸形，胸骨无压痛，乳房正常对称。  双侧呼吸运动对称，呼吸规整，肋间隙无增宽、变窄，双侧语颤正常对称。双肺叩诊清音，双肺下界移动度正常对称。双肺呼吸音清晰，未闻及干湿性啰音，无胸膜摩擦音。  心前区无隆起，心尖搏动位于左第五肋间锁骨中线内0.5cm，无抬举感，无震颤，心相对浊音界无扩大。心率80次/分，律齐，A2>P2，各瓣膜听诊区未闻及杂音，无心包摩擦音。  腹平坦，未见胃形、肠形，无腹壁静脉曲张，腹壁柔软，全腹无压痛、反跳痛，未触及包块，肝脏肋下未触及，脾脏肋下未触及，墨菲氏征阴性。叩诊呈鼓音，肝上界位于右锁骨中线第五肋间，肝区无叩击痛，移动性浊音阴性，双肾区无叩击痛。肠鸣音无亢进或减弱，4次/分，无气过水音，未闻及血管杂音。  肛门及外生殖器无异常。  脊柱生理弯度存在，无侧弯，四肢活动自如，无畸形、杵状指（趾），关节无红肿、变形，双下肢无浮肿。  四肢肌力、肌张力正常对称，腹壁反射存在。双侧肱二、三头肌腱反射正常对称，双侧膝反射、跟腱反射正常对称。双侧霍夫曼氏征阴性，双侧巴氏征阴性，克氏征阴性，布氏征阴性。  神清，精神反应一般，头部可见陈旧手术疤痕，愈合良好。眼睑无下垂，双侧眼结膜无充血水肿，双侧瞳孔等大等圆，对光反射灵敏，呼吸平顺，双肺未闻及罗音。腹软不张。四肢活动自如，双侧膝反射可引出，克氏征，布氏征，巴氏征均为阴性。  头颅MR(2016-06-14，我院)：癫痫术后复查：右枕部部分脑组织缺如；并右顶枕部硬膜下少许出血；右顶枕部头皮肿胀，请结合临床。", "家族史": "见第一次住院病历。", "诊断": "  1.难治性癫痫  2.抑郁综合征  樊轶斌  2016-08-22  第 页  """
text_cut = jieba.posseg.cut(text)

seg_word_dict = {}
for w in text_cut:
    if w.flag in seg_word_dict.keys():
        seg_word_dict[w.flag] += w.word
    else:
        key = w.flag
        value = w.word
        seg_word_dict[key] = value
print(seg_word_dict['x'])

、，、。，，、。  、、、、、、、、、、。  ，、。，，、，，，，。，，，，。，、，，。、，、，、，、、，，。  ，，，，，，，、、。  ，，。  ，，、，。，。，，。  ，，，，。/，，>，，。  ，、，，，、，，，，。，，，，。，/，，。  。  ，，，、（），、，。  、，。、，、。，，，。  ，，，。，，，，，。。，，，，。  (--，)：：；；，。", "": "。", "": "        --     
